In [31]:
import sys
import numpy as np
import math
import scipy
import copy
import scipy.integrate as integrate
from scipy.integrate import quad
import collections
#import matplotlib.pyplot as plt
#from mpl_toolkits.mplot3d import Axes3D
#from matplotlib import ticker, cm
#import seaborn as sns
from numpy import sqrt


#Header Codes

UnitCode = '164'
NodeCoordinates = '2411'
ElementList = '2412'
BoundaryConditions = '2467'

In [16]:
# reading the entire mesh .unv file into a List

dataList=[]
x = open("UT12/mesh.unv","r")
for line in x:
    line=line.rstrip('\n')
    dataList.append(line.lstrip(' '))
dataList.append('end')
x.close()

#print(dataList)

In [17]:
# reading boundary condition file

boundList=[]
x = open("UT12/boundarycondition.dat","r")
for line in x:
    line=line.rstrip('\n')
    boundList.append(line.lstrip(' '))
boundList.append('end')
x.close()

#print(boundList)

In [18]:
#initial condition and heat generation input
initialcond = float(boundList[1])
G = float(boundList[3])
boundarylist=[]
i=5
while boundList[i]!=('end'):
    if i < len(boundList):
        boundarylist.append(boundList[i])
        if(boundList[i+1]=='end'):
            i+=1
        else:
            i+=2
#print(boundarylist)

In [19]:
def check(text,linenumber):
    found = False
    if text in dataList[linenumber]:
        found = True
    return found
        
    
def findBlockStart(dataList,HeaderCode):
    linenumber=0
    for linenumber in range(len(dataList)):
        if check('-1',linenumber)== True:
            if check(HeaderCode,linenumber+1)==True:
                return(linenumber+2)

In [20]:
# code block to create NODE DICTIONARY

NodeDictionary  = {}
startLine = findBlockStart(dataList,NodeCoordinates)

line = copy.deepcopy(startLine)
coordinateLine = line+1

while dataList[line]!=('-1'):
    NodeDictionary.update({int((dataList[line].split())[0]):[float(x) for x in dataList[coordinateLine].split()]})
    line+=2
    coordinateLine+=2

print(NodeDictionary)

{1: [-3.885780586188048e-16, 2.2737367544323206e-15, 0.0], 2: [-3.885780586188048e-16, 0.009000000000002274, 0.0], 3: [0.006249999999999612, 0.009000000000002274, 0.0], 4: [0.006249999999999612, 2.2737367544323206e-15, 0.0], 5: [-3.885780586188048e-16, 0.0005000000000022738, 0.0], 6: [-3.885780586188048e-16, 0.0010000000000022738, 0.0], 7: [-3.885780586188048e-16, 0.0015000000000022738, 0.0], 8: [-3.885780586188048e-16, 0.002000000000002274, 0.0], 9: [-3.885780586188048e-16, 0.002500000000002274, 0.0], 10: [-3.885780586188048e-16, 0.003000000000002274, 0.0], 11: [-3.885780586188048e-16, 0.003500000000002274, 0.0], 12: [-3.885780586188048e-16, 0.0040000000000022734, 0.0], 13: [-3.885780586188048e-16, 0.004500000000002274, 0.0], 14: [-3.885780586188048e-16, 0.005000000000002274, 0.0], 15: [-3.885780586188048e-16, 0.005500000000002275, 0.0], 16: [-3.885780586188048e-16, 0.006000000000002275, 0.0], 17: [-3.885780586188048e-16, 0.006500000000002276, 0.0], 18: [-3.885780586188048e-16, 0.0070

In [21]:
# code block to create Element Dictionary
EdgeElementDictionary = {}
ElementDictionary = {}
elementStart = findBlockStart(dataList,ElementList)

line = copy.deepcopy(elementStart)
i=1
while dataList[line]!=('-1'):
    if [int(x) for x in dataList[line].split()][5] == 2:
        EdgeElementDictionary.update({int((dataList[line].split())[0]):[int(x) for x in dataList[line+2].split()]})
        line+=3
    else:
        ElementDictionary.update({i:[int(x) for x in dataList[line+1].split()]})
        line+=2
        i+=1

#print(EdgeElementDictionary)
print(ElementDictionary)

{1: [73, 20, 21], 2: [126, 261, 22], 3: [67, 114, 113], 4: [114, 18, 19], 5: [124, 73, 255], 6: [19, 254, 114], 7: [113, 254, 73], 8: [117, 74, 115], 9: [116, 117, 25], 10: [115, 24, 25], 11: [75, 117, 116], 12: [115, 25, 117], 13: [119, 118, 74], 14: [120, 67, 118], 15: [76, 120, 119], 16: [118, 119, 120], 17: [122, 121, 75], 18: [123, 76, 121], 19: [65, 123, 122], 20: [121, 122, 123], 21: [119, 121, 76], 22: [117, 75, 121], 23: [74, 117, 119], 24: [121, 119, 117], 25: [113, 118, 67], 26: [124, 74, 118], 27: [73, 124, 113], 28: [118, 113, 124], 29: [24, 115, 125], 30: [21, 261, 73], 31: [74, 125, 115], 32: [125, 23, 24], 33: [136, 80, 256], 34: [86, 158, 267], 35: [124, 255, 125], 36: [126, 22, 23], 37: [74, 124, 125], 38: [23, 125, 255], 39: [129, 77, 127], 40: [128, 129, 30], 41: [127, 29, 30], 42: [78, 129, 128], 43: [127, 30, 129], 44: [131, 130, 77], 45: [132, 65, 130], 46: [79, 132, 131], 47: [130, 131, 132], 48: [134, 133, 78], 49: [135, 79, 133], 50: [68, 135, 134], 51: [133, 

In [22]:
# Read Boundaries
TempBDictionary={}
Convectbound={}
qbound={}
rbound={}
boundaryConditionStart = findBlockStart(dataList,BoundaryConditions)
line = copy.deepcopy(boundaryConditionStart)
while dataList[line]!=('-1'):
    btot = int((dataList[line].split())[7])
    bname = (dataList[line+1].split())[0]
    for i in range(len(boundarylist)):
        if bname == (boundarylist[i].split())[0]:
            if (boundarylist[i].split())[1] == 'Temp':
                for xi in range(int(btot/2+btot%2)-1):
                    k = line+1+xi+1
                    TempBDictionary.update({int((dataList[k].split())[1]):float((boundarylist[i].split())[2])})
                    TempBDictionary.update({int((dataList[k].split())[5]):float((boundarylist[i].split())[2])})
                if int(btot%2)==1:
                    j = line+1+int(btot/2+btot%2
                                  )
                    TempBDictionary.update({int((dataList[j].split())[1]):float((boundarylist[i].split())[2])})
                else:
                    j = line+1+int(btot/2+btot%2)
                    TempBDictionary.update({int((dataList[j].split())[1]):float((boundarylist[i].split())[2])})
                    TempBDictionary.update({int((dataList[j].split())[5]):float((boundarylist[i].split())[2])})
            if (boundarylist[i].split())[1] == 'Conv':
                for xi in range(int(btot/2+btot%2)-1):
                    k = line+1+xi+1
                    Convectbound.update({int((dataList[k].split())[1]):[float((boundarylist[i].split())[2]),float((boundarylist[i].split())[3])]})
                    Convectbound.update({int((dataList[k].split())[5]):[float((boundarylist[i].split())[2]),float((boundarylist[i].split())[3])]})
                if int(btot%2)==1:
                    j = line+1+int(btot/2+btot%2)
                    Convectbound.update({int((dataList[j].split())[1]):[float((boundarylist[i].split())[2]),float((boundarylist[i].split())[3])]})
                else:
                    j = line+1+int(btot/2+btot%2)
                    Convectbound.update({int((dataList[j].split())[1]):[float((boundarylist[i].split())[2]),float((boundarylist[i].split())[3])]})
                    Convectbound.update({int((dataList[j].split())[5]):[float((boundarylist[i].split())[2]),float((boundarylist[i].split())[3])]})
            if (boundarylist[i].split())[1] == 'Radi':
                for xi in range(int(btot/2+btot%2)-1):
                    k = line+1+xi+1
                    rbound.update({int((dataList[k].split())[1]):[float((boundarylist[i].split())[2]),float((boundarylist[i].split())[3])]})
                    rbound.update({int((dataList[k].split())[5]):[float((boundarylist[i].split())[2]),float((boundarylist[i].split())[3])]})
                if int(btot%2)==1:
                    j = line+1+int(btot/2+btot%2)
                    rbound.update({int((dataList[j].split())[1]):[float((boundarylist[i].split())[2]),float((boundarylist[i].split())[3])]})
                else:
                    j = line+1+int(btot/2+btot%2)
                    rbound.update({int((dataList[j].split())[1]):[float((boundarylist[i].split())[2]),float((boundarylist[i].split())[3])]})
                    rbound.update({int((dataList[j].split())[5]):[float((boundarylist[i].split())[2]),float((boundarylist[i].split())[3])]})
            if (boundarylist[i].split())[1] == 'Flux':
                for xi in range(int(btot/2+btot%2)-1):
                    k = line+1+xi+1
                    qbound.update({int((dataList[k].split())[1]):float((boundarylist[i].split())[2])})
                    qbound.update({int((dataList[k].split())[5]):float((boundarylist[i].split())[2])})
                if int(btot%2)==1:
                    j = line+1+int(btot/2+btot%2)
                    qbound.update({int((dataList[j].split())[1]):float((boundarylist[i].split())[2])})
                else:
                    j = line+1+int(btot/2+btot%2)
                    qbound.update({int((dataList[j].split())[1]):float((boundarylist[i].split())[2])})
                    qbound.update({int((dataList[j].split())[5]):float((boundarylist[i].split())[2])})
    line+=int(btot/2+btot%2)+2
#print(TempBDictionary)
#print(Convectbound)
#print(rbound)
print(qbound)

{3: 20000.0, 4: 20000.0, 34: 20000.0, 35: 20000.0, 36: 20000.0, 37: 20000.0, 38: 20000.0, 39: 20000.0, 40: 20000.0, 41: 20000.0, 42: 20000.0, 43: 20000.0, 44: 20000.0, 45: 20000.0, 46: 20000.0, 47: 20000.0, 48: 20000.0, 49: 20000.0, 50: 20000.0}


In [23]:
#reading material dataA
# reading boundary condition file

materialList=[]
x = open("UT12/material.dat","r")
for line in x:
    line=line.rstrip('\n')
    materialList.append(line.lstrip(' '))
materialList.append('end')
x.close()
density = float(materialList[1])
specificheat = float(materialList[3])
k = float(materialList[5])
print(density)
print(specificheat)
print(k)

8470.0
444.0
14.9


In [24]:
#calculate smaller element length
length=[]
for x in range(len(ElementDictionary)):
    length.append(math.sqrt((NodeDictionary[ElementDictionary[x+1][0]][0]-NodeDictionary[ElementDictionary[x+1][1]][0])**2+(NodeDictionary[ElementDictionary[x+1][0]][1]-NodeDictionary[ElementDictionary[x+1][1]][1])**2))
    length.append(math.sqrt((NodeDictionary[ElementDictionary[x+1][1]][0]-NodeDictionary[ElementDictionary[x+1][2]][0])**2+(NodeDictionary[ElementDictionary[x+1][1]][1]-NodeDictionary[ElementDictionary[x+1][2]][1])**2))
    length.append(math.sqrt((NodeDictionary[ElementDictionary[x+1][2]][0]-NodeDictionary[ElementDictionary[x+1][0]][0])**2+(NodeDictionary[ElementDictionary[x+1][2]][1]-NodeDictionary[ElementDictionary[x+1][0]][1])**2))
#calculate time step
alpha = k/(density*specificheat)
timestep = min(length)**2/(2*alpha)
timestep

0.0045620551368483015

In [25]:
#Type of Analysis 2D or axisymmetric
analysis = 'axisymmetric'

#C, K and F Matrix with assembly
C = np.zeros((len(NodeDictionary),len(NodeDictionary)))
K = np.zeros((len(NodeDictionary),len(NodeDictionary)))
F = np.zeros((len(NodeDictionary),1))
Fpart=[]
nele = len(ElementDictionary)


#for loop for each element
for x in range(nele):
    #element area
    elemarea = abs((np.linalg.det(np.array([[1,NodeDictionary[ElementDictionary[x+1][0]][0],
                               NodeDictionary[ElementDictionary[x+1][0]][1]],
                              [1,NodeDictionary[ElementDictionary[x+1][1]][0],
                               NodeDictionary[ElementDictionary[x+1][1]][1]],
                              [1,NodeDictionary[ElementDictionary[x+1][2]][0],
                               NodeDictionary[ElementDictionary[x+1][2]][1]]])))/2)
    
    #shape function constants a,b and c
    ai = NodeDictionary[ElementDictionary[x+1][1]][0]*NodeDictionary[ElementDictionary[x+1][2]][1] - NodeDictionary[ElementDictionary[x+1][2]][0]*NodeDictionary[ElementDictionary[x+1][1]][1]
    bi = NodeDictionary[ElementDictionary[x+1][1]][1] - NodeDictionary[ElementDictionary[x+1][2]][1]
    ci = NodeDictionary[ElementDictionary[x+1][2]][0] - NodeDictionary[ElementDictionary[x+1][1]][0]
    aj = NodeDictionary[ElementDictionary[x+1][2]][0]*NodeDictionary[ElementDictionary[x+1][0]][1] - NodeDictionary[ElementDictionary[x+1][0]][0]*NodeDictionary[ElementDictionary[x+1][2]][1]
    bj = NodeDictionary[ElementDictionary[x+1][2]][1] - NodeDictionary[ElementDictionary[x+1][0]][1]
    cj = NodeDictionary[ElementDictionary[x+1][0]][0] - NodeDictionary[ElementDictionary[x+1][2]][0]
    ak = NodeDictionary[ElementDictionary[x+1][0]][0]*NodeDictionary[ElementDictionary[x+1][1]][1] - NodeDictionary[ElementDictionary[x+1][1]][0]*NodeDictionary[ElementDictionary[x+1][0]][1]
    bk = NodeDictionary[ElementDictionary[x+1][0]][1] - NodeDictionary[ElementDictionary[x+1][1]][1]
    ck = NodeDictionary[ElementDictionary[x+1][1]][0] - NodeDictionary[ElementDictionary[x+1][0]][0]
   
    #B, D and BTranspose Matrix and multiplication
    B = (1/(2*elemarea))*np.matrix([[bi,bj,bk], [ci,cj,ck]])
    D = np.array([[k,0],[0,k]])
    BTranspose = np.array(B).transpose()
    Bmult = np.matmul(np.matmul(BTranspose,D),B)
    
    if analysis == 'axisymmetric':
        #average r value
        raverage = (NodeDictionary[ElementDictionary[x+1][0]][0]+NodeDictionary[ElementDictionary[x+1][1]][0]+NodeDictionary[ElementDictionary[x+1][2]][0])/3
        Bmult = Bmult * 2 * np.pi * raverage
        
    # calculating the first half of the K matrix
    K1 = np.zeros(Bmult.shape)
    for i in range(list(Bmult.shape)[0]): 
        for j in range(list(Bmult.shape)[1]):
            K1[i,j]= list(quad(lambda x:(Bmult.item(i,j)),0,elemarea))[0]
    h=0  
    lij=0
    
    # calculating the second half of the K matrix
    K2 = np.zeros(Bmult.shape)
    for i in range(list(Bmult.shape)[0]): 
        for j in range(list(Bmult.shape)[1]):
            if ElementDictionary[x+1][i] in Convectbound:
                h = Convectbound[ElementDictionary[x+1][i]][0]
                if ElementDictionary[x+1][j] in Convectbound:
                    if i==j:
                        K2[i,j]= 2
                    else:
                        K2[i,j]= 1
                        # length of the element in convective boundary region
                        lij = np.sqrt((NodeDictionary[ElementDictionary[x+1][i]][0]-
                                       NodeDictionary[ElementDictionary[x+1][j]][0])**2+
                                      (NodeDictionary[ElementDictionary[x+1][i]][1]-
                                       NodeDictionary[ElementDictionary[x+1][j]][1])**2)
    Ke = K1+K2*h*lij/6
    
    #K matrix assembly
    for i in range(list(Ke.shape)[0]): # calculating the first half of the integral
        for j in range(list(Ke.shape)[1]):
            K[int(ElementDictionary[(x+1)][i])-1,int(ElementDictionary[(x+1)][j])-1]+=Ke[i,j]
    
    #C matrix
    ctemp = np.zeros((3,3))
    for i in range(3): 
        for j in range(3):
            if i==j:
                if analysis == 'axisymmetric':
                    ctemp[i,j]= 2*elemarea*density*specificheat*2*np.pi * raverage/12
                else:
                    ctemp[i,j]= 2*elemarea*density*specificheat/12                         
            else:
                if analysis == 'axisymmetric':
                    ctemp[i,j]= 1*elemarea*density*specificheat*2*np.pi * raverage/12
                else:
                    ctemp[i,j]= 1*elemarea*density*specificheat/12
            
    #C matrix assembly
    for l in range(3): # calculating the first half of the integral
        for m in range(3):
            C[int(ElementDictionary[(x+1)][l])-1,int(ElementDictionary[(x+1)][m])-1]+=ctemp[l,m]
    #print(C)
    
    #F matrix
    F1=(G*elemarea/3)*np.matrix([[1],[1],[1]])  #F1 is for internal heat generation part
    
    #F2 is for heat flux part
    F2=np.zeros((3,1))
    p1=np.zeros((3,1))
    F2p1 =[0,0]
    F2Lij=0
    j=0
    q=0
    h=0
    Ta=0
    for i in range(3):
        if ElementDictionary[x+1][i] in qbound:
            q = qbound[ElementDictionary[x+1][i]]
            F2p1[j]=ElementDictionary[x+1][i]
            p1[i][0]+=1
            j+=1
    if (F2p1[0]!=0)&(F2p1[1]!=0):
        F2lij = np.sqrt((NodeDictionary[F2p1[0]][0]-
                                      NodeDictionary[F2p1[1]][0])**2+
                                      (NodeDictionary[F2p1[0]][1]-
                                      NodeDictionary[F2p1[1]][1])**2)
        if analysis == 'axisymmetric':
            for i in range(3):
                if p1[i][0]==1:
                    p1[i][0]=2*NodeDictionary[ElementDictionary[x+1][i]][0]
            if (p1[0][0]>0) & (p1[1][0]>0): 
                p1[0][0]+=NodeDictionary[ElementDictionary[x+1][1]][0]
                p1[1][0]+=NodeDictionary[ElementDictionary[x+1][0]][0]
            elif (p1[1][0]>0) & (p1[2][0]>0):
                p1[1][0]+=NodeDictionary[ElementDictionary[x+1][2]][0]
                p1[2][0]+=NodeDictionary[ElementDictionary[x+1][1]][0]
            elif (p1[0][0]>0) & (p1[2][0]>0):
                p1[0][0]+=NodeDictionary[ElementDictionary[x+1][2]][0]
                p1[2][0]+=NodeDictionary[ElementDictionary[x+1][0]][0]
            F2=(2*np.pi*q*F2lij/6)*p1
        else:
            F2=(q*F2lij/2)*p1
    
    #F3 is for convective part
    F3=np.zeros((3,1))
    p1=np.zeros((3,1))
    F2p1 =[0,0]
    F2Lij=0
    m=0
    for i in range(3):
        if ElementDictionary[x+1][i] in Convectbound:
            h = Convectbound[ElementDictionary[x+1][i]][0]
            Ta = Convectbound[ElementDictionary[x+1][i]][1]
            F2p1[m]=ElementDictionary[x+1][i]
            p1[i][0]+=1
            m+=1
    if (F2p1[0]!=0)&(F2p1[1]!=0):
        F2lij = np.sqrt((NodeDictionary[F2p1[0]][0]-
                                      NodeDictionary[F2p1[1]][0])**2+
                                      (NodeDictionary[F2p1[0]][1]-
                                      NodeDictionary[F2p1[1]][1])**2)
        F3=(h*Ta*F2lij/2)*p1
    # Total Load matrix F
    Fe =F1+F2+F3
    Fpart.append(F1+F3)
    
    #Assembly of F Matrix
    for i in range(list(Fe.shape)[0]):
        F[int(ElementDictionary[(x+1)][i])-1,0]+=Fe[i,0]
print(F)

[[0.        ]
 [0.        ]
 [0.19634954]
 [0.19634954]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.39269908]
 [0.39269908]
 [0.39269908]
 [0.39269908]
 [0.39269908]
 [0.39269908]
 [0.39269908]
 [0.39269908]
 [0.39269908]
 [0.39269908]
 [0.39269908]
 [0.39269908]
 [0.39269908]
 [0.39269908]
 [0.39269908]
 [0.39269908]
 [0.39269908]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.  

# Read temperature input file

In [26]:
Tmeasured=[]
x = open("UT12/output.txt","r")
for line in x:
    line=line.rstrip('\n')
    Tmeasured.append(line.lstrip(' '))
x.close()
Tmeasured

['851',
 '848.5',
 '838',
 '819',
 '795.5',
 '771',
 '746',
 '722.5',
 '701',
 '683.5',
 '667.5',
 '652',
 '638',
 '622',
 '593',
 '553',
 '514',
 '481',
 '454',
 '433',
 '417',
 '403.5',
 '392.5',
 '382.5',
 '373.5',
 '365',
 '357',
 '349.5',
 '342',
 '335',
 '328',
 '321',
 '314.5',
 '308.5',
 '302',
 '296',
 '290',
 '284.5',
 '279',
 '273.5',
 '268',
 '263',
 '258',
 '253.5',
 '249',
 '244',
 '239.5',
 '235',
 '231',
 '226.5',
 '222.5',
 '218',
 '214',
 '209.5',
 '205.5',
 '202',
 '198.5',
 '195',
 '191.5',
 '188.5',
 '185',
 '182',
 '179',
 '175.5',
 '172.5',
 '170',
 '167',
 '164.5',
 '162',
 '159.5',
 '157',
 '155',
 '152.5',
 '150.5',
 '148',
 '146',
 '144',
 '142',
 '140',
 '138',
 '136.5',
 '134.5',
 '132.5',
 '131',
 '129',
 '127.5',
 '126',
 '124.5',
 '123',
 '121.5',
 '120',
 '118.5',
 '117',
 '115.5',
 '114',
 '113',
 '111.5',
 '110',
 '109',
 '108',
 '106.5',
 '105.5',
 '104.5',
 '103',
 '102',
 '101',
 '100',
 '99',
 '98',
 '97',
 '96',
 '95',
 '94',
 '93',
 '92.5',
 '91

# Genetic Algorithm

In [46]:
import random 
  
# Number of individuals in each generation 
POPULATION_SIZE = 50
  
# Valid genes 
Max_GENE = 1000000
Min_GENE = 0
timestep=1
theta = 1
# Target string to be generated 
#TARGET = 99.9799

class Individual(object): 
    ''' 
    Class representing individual in population 
    '''
    def __init__(self, chromosome): 
        self.chromosome = chromosome  
        self.fitness = self.cal_fitness() 
  
    @classmethod
    def mutated_genes(self): 
        ''' 
        create random genes for mutation 
        '''
        global Max_GENE
        global Min_GENE
        gene = random.uniform(Min_GENE, Max_GENE)
        return gene 
  
    @classmethod
    def create_gnome(self): 
        ''' 
        create chromosome or string of genes 
        '''
        #global TARGET 
        #gnome_len = len(TARGET) 
        #return [self.mutated_genes() for _ in range(gnome_len)] 
        return self.mutated_genes()
  
    def mate(self, par2): 
        ''' 
        Perform mating and produce new offspring 
        '''
  
        # chromosome for offspring 
        child_chromosome = []     
  
        # random probability   
        prob = random.random() 
  
        # if prob is less than 0.45, insert gene 
        # from parent 1  
        if prob < 0.20: 
            child_chromosome.append(self.chromosome*2) 
        elif prob < 0.40: 
            child_chromosome.append(self.chromosome+random.uniform(-5000, 5000)) 
        elif prob < 0.60: 
            child_chromosome.append(self.chromosome/2)  
        # if prob is between 0.45 and 0.90, insert 
        # gene from parent 2 
        elif prob < 0.80: 
            child_chromosome.append(par2.chromosome+random.uniform(-1000, 1000)) 
        elif prob < 0.90: 
            child_chromosome.append(par2.chromosome+random.uniform(-200, 200)) 
        elif prob < 0.98: 
            child_chromosome.append(par2.chromosome+random.uniform(-1, 1)) 
        # otherwise insert random gene(mutate),  
        # for maintaining diversity 
        else: 
            child_chromosome.append(self.mutated_genes()) 
        # create new Individual(offspring) using  
        # generated chromosome for offspring 
        return Individual(child_chromosome[0])
  
    def cal_fitness(self): 
        ''' 
        Calculate fittness score, it is the number of 
        characters in string which differ from target 
        string. 
        '''
        global TARGET 
        fitness = 0
        #print(self.chromosome)
        flx = float(self.chromosome)
        F = np.zeros((len(NodeDictionary),1))
        nele = len(ElementDictionary)
        #for loop for each element
        for x in range(nele):
            F2=np.zeros((3,1))
            p1=np.zeros((3,1))
            F2p1 =[0,0]
            F2Lij=0
            j=0
            q=0
            h=0
            Ta=0
            for i in range(3):
                if ElementDictionary[x+1][i] in qbound:
                    q = -flx
                    F2p1[j]=ElementDictionary[x+1][i]
                    p1[i][0]+=1
                    j+=1
            if (F2p1[0]!=0)&(F2p1[1]!=0):
                F2lij = np.sqrt((NodeDictionary[F2p1[0]][0]-
                                              NodeDictionary[F2p1[1]][0])**2+
                                              (NodeDictionary[F2p1[0]][1]-
                                              NodeDictionary[F2p1[1]][1])**2)
                if analysis == 'axisymmetric':
                    for i in range(3):
                        if p1[i][0]==1:
                            p1[i][0]=2*NodeDictionary[ElementDictionary[x+1][i]][0]
                    if (p1[0][0]>0) & (p1[1][0]>0): 
                        p1[0][0]+=NodeDictionary[ElementDictionary[x+1][1]][0]
                        p1[1][0]+=NodeDictionary[ElementDictionary[x+1][0]][0]
                    elif (p1[1][0]>0) & (p1[2][0]>0):
                        p1[1][0]+=NodeDictionary[ElementDictionary[x+1][2]][0]
                        p1[2][0]+=NodeDictionary[ElementDictionary[x+1][1]][0]
                    elif (p1[0][0]>0) & (p1[2][0]>0):
                        p1[0][0]+=NodeDictionary[ElementDictionary[x+1][2]][0]
                        p1[2][0]+=NodeDictionary[ElementDictionary[x+1][0]][0]
                    F2=(2*np.pi*q*F2lij/6)*p1
                else:
                    F2=(q*F2lij/2)*p1
            
            Fe =Fpart[x]+F2
            for i in range(list(Fe.shape)[0]):
                F[int(ElementDictionary[(x+1)][i])-1,0]+=Fe[i,0]

        A=np.matrix(C)-(1-theta)*timestep*np.matrix(K)
        RHS=(np.matmul(A,Tem))+timestep*(F)
        TMat=np.zeros((len(NodeDictionary),1))
        for j in NodeDictionary:
            if j in TempBDictionary:
                RHS[j-1][0]=TempBDictionary[j]
                TMat[j-1][0]=TempBDictionary[j]
        CTemp = np.matrix(C)+theta*timestep*K
        for z in range(len(TMat)):
            if TMat[z]!=0 :
                CTemp[z,:]=0
                CTemp[z,z]=1
        TMatFin = np.linalg.solve(CTemp,RHS)
        Tpredicted = float(TMatFin[12])
        Tmeasu = float(TARGET)
        diff = abs(Tpredicted - Tmeasu)
        fitness = 1/(1+diff)
        #print(fitness)
        #print(TMatFin[75])
        return fitness

In [47]:
# Driver code
steps=5
initial = 0
def ga(Tm,Te): 
    global POPULATION_SIZE 
    global TARGET
    global Tem
    
    #current generation 
    generation = 1
    TARGET = Tm
    Tem = Te
    found = False
    population = []
    # create initial population 
    for _ in range(POPULATION_SIZE): 
                gnome = Individual.create_gnome()
                population.append(Individual(gnome)) 
    
    while not found: 
        # sort the population in increasing order of fitness score 
        population = sorted(population, key = lambda x:x.fitness, reverse=True) 
  
        # if the individual having lowest fitness score ie.  
        # 0 then we know that we have reached to the target 
        # and break the loop 
        if generation < 100:
            if population[0].fitness > 0.999: 
                found = True
                break
        else:
            if generation == 250:
                found = True
                break
            else:
                if population[0].fitness > 0.995:
                    found = True
                    break
  
        # Otherwise generate new offsprings for new generation 
        new_generation = [] 
  
        # Perform Elitism, that mean 10% of fittest population 
        # goes to the next generation 
        s = int((10*POPULATION_SIZE)/100) 
        new_generation.extend(population[:s]) 
  
        # From 50% of fittest population, Individuals  
        # will mate to produce offspring 
        s = int((90*POPULATION_SIZE)/100) 
        for _ in range(s): 
            parent1 = random.choice(population[:s]) 
            parent2 = random.choice(population[:s]) 
            child = parent1.mate(parent2) 
            new_generation.append(child) 
  
        population = new_generation 
  
        print("Generation: {}\tString: {}\tFitness: {}".\
              format(generation, 
              population[0].chromosome, 
              population[0].fitness)) 
  
        generation += 1 
    print("Generation: {}\tString: {}\tFitness: {}".\
          format(generation, 
          population[0].chromosome, 
          population[0].fitness)) 
    return (population[0].chromosome)
import time
start_time = time.time()
Tem=np.zeros((len(NodeDictionary),1))
for i in range(len(Tem)):
    Tem[i-1]+=851
T75 = []
HeatFlux = []
for st in range(steps):
    HFtemp = ga(Tmeasured[st],Tem)
    HeatFlux.append(HFtemp)
    F = np.zeros((len(NodeDictionary),1))
    nele = len(ElementDictionary)
    #for loop for each element
    for x in range(nele):
            F2=np.zeros((3,1))
            p1=np.zeros((3,1))
            F2p1 =[0,0]
            F2Lij=0
            j=0
            q=0
            h=0
            Ta=0
            for i in range(3):
                if ElementDictionary[x+1][i] in qbound:
                    q = -HFtemp
                    F2p1[j]=ElementDictionary[x+1][i]
                    p1[i][0]+=1
                    j+=1
            if (F2p1[0]!=0)&(F2p1[1]!=0):
                F2lij = np.sqrt((NodeDictionary[F2p1[0]][0]-
                                              NodeDictionary[F2p1[1]][0])**2+
                                              (NodeDictionary[F2p1[0]][1]-
                                              NodeDictionary[F2p1[1]][1])**2)
                if analysis == 'axisymmetric':
                    for i in range(3):
                        if p1[i][0]==1:
                            p1[i][0]=2*NodeDictionary[ElementDictionary[x+1][i]][0]
                    if (p1[0][0]>0) & (p1[1][0]>0): 
                        p1[0][0]+=NodeDictionary[ElementDictionary[x+1][1]][0]
                        p1[1][0]+=NodeDictionary[ElementDictionary[x+1][0]][0]
                    elif (p1[1][0]>0) & (p1[2][0]>0):
                        p1[1][0]+=NodeDictionary[ElementDictionary[x+1][2]][0]
                        p1[2][0]+=NodeDictionary[ElementDictionary[x+1][1]][0]
                    elif (p1[0][0]>0) & (p1[2][0]>0):
                        p1[0][0]+=NodeDictionary[ElementDictionary[x+1][2]][0]
                        p1[2][0]+=NodeDictionary[ElementDictionary[x+1][0]][0]
                    F2=(2*np.pi*q*F2lij/6)*p1
                else:
                    F2=(q*F2lij/2)*p1
            Fe =Fpart[x]+F2
            for i in range(list(Fe.shape)[0]):
                F[int(ElementDictionary[(x+1)][i])-1,0]+=Fe[i,0]
    A=np.matrix(C)-(1-theta)*timestep*np.matrix(K)
    RHS=(np.matmul(A,Tem))+timestep*(F)
    TMat=np.zeros((len(NodeDictionary),1))
    for j in NodeDictionary:
        if j in TempBDictionary:
            RHS[j-1][0]=TempBDictionary[j]
            TMat[j-1][0]=TempBDictionary[j]
    CTemp = np.matrix(C)+theta*timestep*K
    for z in range(len(TMat)):
        if TMat[z]!=0 :
            CTemp[z,:]=0
            CTemp[z,z]=1
    TMatFin = np.linalg.solve(CTemp,RHS)
    Tem = TMatFin
    print(TMatFin[12])
    print('Iteration:'+str(st+1))
    T75.append(float(TMatFin[12]))
print("--- %s seconds ---" % (time.time() - start_time))

Generation: 1	String: 3637.790278228081	Fitness: 0.9040383157133022
Generation: 2	String: 3420.068553754003	Fitness: 0.9092604770307835
Generation: 3	String: 3420.068553754003	Fitness: 0.9092604770307835
Generation: 4	String: 3411.968881143675	Fitness: 0.909455915834901
Generation: 5	String: 1818.8951391140406	Fitness: 0.9496009699517783
Generation: 6	String: -666.8475998169088	Fitness: 0.980913311446771
Generation: 7	String: -333.4237999084544	Fitness: 0.9903647027644938
Generation: 8	String: -333.4237999084544	Fitness: 0.9903647027644938
Generation: 9	String: 232.352385108102	Fitness: 0.993265800414872
Generation: 10	String: 98.88997803056998	Fitness: 0.9971227719770841
Generation: 11	String: 98.88997803056998	Fitness: 0.9971227719770841
Generation: 12	String: -83.60209320271505	Fitness: 0.9975664942784899
Generation: 13	String: -83.60209320271505	Fitness: 0.9975664942784899
Generation: 14	String: -76.7840041833683	Fitness: 0.9977645128580459
Generation: 15	String: 15.80657099852047	

In [ ]:
HeatFlux

In [29]:
T75

[500.4723170664634,
 500.8219817880538,
 501.3832499316187,
 502.1694204521384,
 502.7115702347761,
 503.533937776988,
 504.07182145995347,
 505.3773854700976,
 506.7048265563585,
 509.9408958440795,
 513.9814650579278,
 519.4760484784267,
 528.1974015719054,
 539.82892794102,
 569.5616368500644,
 618.0462464862524,
 672.380073293595,
 716.8509539450779,
 755.825975527988,
 794.6814089961937,
 832.7848624611736,
 870.9079948545343,
 907.7732162663171,
 943.9307259782819,
 978.4185712633645,
 1013.8464944442866,
 1047.5314967411068,
 1080.1480516879146,
 1111.7546312530765,
 1142.224189099403,
 1172.258549078181,
 1201.2349847457506,
 1229.291652425739,
 1256.6738155861694,
 1282.8686008294098,
 1307.7922089238357,
 1331.8342016399072,
 1354.5293256748112,
 1376.1507904476946,
 1398.6997146021524,
 1413.302132194323,
 1459.9792460255471,
 1493.2407915316228,
 1513.7556674231598,
 1530.799325129128,
 1546.5755124175087,
 1560.401289426144,
 1574.046411663656,
 1587.7930278002907,
 1601.1

In [ ]:
#Create individual
from gaft.components import BinaryIndividual
indv = BinaryIndividual(ranges=[(0, 1000000000)], eps=0.001)

#Create a population with 50 individuals
from gaft.components import Population
population = Population(indv_template=indv, size=50).init()

#Tournament selection
from gaft.operators import TournamentSelection
selection = TournamentSelection()

#Uniform crossover
from gaft.operators import UniformCrossover
crossover = UniformCrossover(pc=0.8, pe=0.5)

#Flip bit mutation
from gaft.operators import FlipBitMutation
mutation = FlipBitMutation(pm=0.1)

#Import an on-the-fly analysis plugin to output info to console
from gaft.analysis import ConsoleOutput

#Create an engine to run
from gaft import GAEngine

timestep=0.1
theta = 0.5

import time
start_time = time.time()
def ga(Tm, Tem):
    engine = GAEngine(population=population, selection=selection,
                      crossover=crossover, mutation=mutation,
                      analysis=[ConsoleOutput])

    #Define target function to optimize
    @engine.fitness_register
    @engine.dynamic_linear_scaling(target='max', ksi0=2, r=0.9)
    @engine.minimize
    def fitness(indv):
        flx, = indv.solution

        F = np.zeros((len(NodeDictionary),1))
        nele = len(ElementDictionary)
        #for loop for each element
        for x in range(nele):
            F2=np.zeros((3,1))
            p1=np.zeros((3,1))
            F2p1 =[0,0]
            F2Lij=0
            j=0
            q=0
            h=0
            Ta=0
            for i in range(3):
                if ElementDictionary[x+1][i] in qbound:
                    q = flx
                    F2p1[j]=ElementDictionary[x+1][i]
                    p1[i][0]+=1
                    j+=1
            if (F2p1[0]!=0)&(F2p1[1]!=0):
                F2lij = np.sqrt((NodeDictionary[F2p1[0]][0]-
                                              NodeDictionary[F2p1[1]][0])**2+
                                              (NodeDictionary[F2p1[0]][1]-
                                              NodeDictionary[F2p1[1]][1])**2)
                F2=(q*F2lij/2)*p1
            Fe =Fpart[x]+F2
            for i in range(list(Fe.shape)[0]):
                F[int(ElementDictionary[(x+1)][i])-1,0]+=Fe[i,0]

        #timestep=2
        #theta = 0.5
        A=np.matrix(C)-(1-theta)*timestep*np.matrix(K)
        RHS=(np.matmul(A,Tem))+timestep*(F)
        TMat=np.zeros((len(NodeDictionary),1))
        for j in NodeDictionary:
            if j in TempBDictionary:
                RHS[j-1][0]=TempBDictionary[j]
                TMat[j-1][0]=TempBDictionary[j]
        CTemp = np.matrix(C)+theta*timestep*K
        for z in range(len(TMat)):
            if TMat[z]!=0 :
                CTemp[z,:]=0
                CTemp[z,z]=1
        TMatFin = np.linalg.solve(CTemp,RHS)
        Tpredicted = float(TMatFin[43])
        Tmeasu = float(Tm)
        return (Tpredicted - Tmeasu)**2
    
    #Run the engine 20 generations
    engine.run(ng=50)

    #Get the best individual
    best_indv = engine.population.best_indv(engine.fitness)

    #Get the solution
    return (best_indv.solution[0])

steps=400
Tem=np.zeros((len(NodeDictionary),1))
for i in range(len(Tem)):
    Tem[i-1]+=851
T75 = []
HeatFlux = []
for st in range(steps+1):
    HFtemp = ga(Tmeasured[st],Tem)
    HeatFlux.append(HFtemp)
    F = np.zeros((len(NodeDictionary),1))
    nele = len(ElementDictionary)
    #for loop for each element
    for x in range(nele):
            F2=np.zeros((3,1))
            p1=np.zeros((3,1))
            F2p1 =[0,0]
            F2Lij=0
            j=0
            q=0
            h=0
            Ta=0
            for i in range(3):
                if ElementDictionary[x+1][i] in qbound:
                    q = HFtemp
                    F2p1[j]=ElementDictionary[x+1][i]
                    p1[i][0]+=1
                    j+=1
            if (F2p1[0]!=0)&(F2p1[1]!=0):
                F2lij = np.sqrt((NodeDictionary[F2p1[0]][0]-
                                              NodeDictionary[F2p1[1]][0])**2+
                                              (NodeDictionary[F2p1[0]][1]-
                                              NodeDictionary[F2p1[1]][1])**2)
                F2=(q*F2lij/2)*p1
            Fe =Fpart[x]+F2
            for i in range(list(Fe.shape)[0]):
                F[int(ElementDictionary[(x+1)][i])-1,0]+=Fe[i,0]
    #timestep=2
    #theta = 0.5
    A=np.matrix(C)-(1-theta)*timestep*np.matrix(K)
    RHS=(np.matmul(A,Tem))+timestep*(F)
    TMat=np.zeros((len(NodeDictionary),1))
    for j in NodeDictionary:
        if j in TempBDictionary:
            RHS[j-1][0]=TempBDictionary[j]
            TMat[j-1][0]=TempBDictionary[j]
    CTemp = np.matrix(C)+theta*timestep*K
    for z in range(len(TMat)):
        if TMat[z]!=0 :
            CTemp[z,:]=0
            CTemp[z,z]=1
    TMatFin = np.linalg.solve(CTemp,RHS)
    Tem = TMatFin
    T75.append(float(TMatFin[43]))
print("--- %s seconds ---" % (time.time() - start_time))

gaft.ConsoleOutput   INFO     Generation number: 50 Population number: 50
gaft.ConsoleOutput   INFO     Generation: 1, best fitness: -121121.599, scaled fitness: 1582.424
gaft.ConsoleOutput   INFO     Generation: 2, best fitness: -121096.491, scaled fitness: 1031.532
gaft.ConsoleOutput   INFO     Generation: 3, best fitness: -121096.491, scaled fitness: 475.547
gaft.ConsoleOutput   INFO     Generation: 4, best fitness: -121096.491, scaled fitness: 458.639
gaft.ConsoleOutput   INFO     Generation: 5, best fitness: -121096.491, scaled fitness: 927.439
gaft.ConsoleOutput   INFO     Generation: 6, best fitness: -121096.430, scaled fitness: 1168.723
gaft.ConsoleOutput   INFO     Generation: 7, best fitness: -121096.430, scaled fitness: 460.239
gaft.ConsoleOutput   INFO     Generation: 8, best fitness: -121096.359, scaled fitness: 224.416
gaft.ConsoleOutput   INFO     Generation: 9, best fitness: -121096.322, scaled fitness: 892.595
gaft.ConsoleOutput   INFO     Generation: 10, best fitness:

gaft.ConsoleOutput   INFO     Generation: 34, best fitness: -121901.520, scaled fitness: 341.631
gaft.ConsoleOutput   INFO     Generation: 35, best fitness: -121901.520, scaled fitness: 1603.407
gaft.ConsoleOutput   INFO     Generation: 36, best fitness: -121901.520, scaled fitness: 146.005
gaft.ConsoleOutput   INFO     Generation: 37, best fitness: -121901.520, scaled fitness: 556.542
gaft.ConsoleOutput   INFO     Generation: 38, best fitness: -121901.520, scaled fitness: 1562.305
gaft.ConsoleOutput   INFO     Generation: 39, best fitness: -121901.520, scaled fitness: 1343.002
gaft.ConsoleOutput   INFO     Generation: 40, best fitness: -121901.520, scaled fitness: 222.586
gaft.ConsoleOutput   INFO     Generation: 41, best fitness: -121901.520, scaled fitness: 445.995
gaft.ConsoleOutput   INFO     Generation: 42, best fitness: -121901.520, scaled fitness: 228.123
gaft.ConsoleOutput   INFO     Generation: 43, best fitness: -121901.520, scaled fitness: 252.463
gaft.ConsoleOutput   INFO  

In [ ]:
random.randrange(-20, 40)